## Clustering Toronto neighborhood
#### In this lab we deal with data about neighborhood, explore technicians оf finding, cleaning, vizualizing and, finally, clustering data wit K-mean method.
#### First import necessary module and library

In [40]:
import numpy as np 
import pandas as pd
import requests
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
!pip install beautifulsoup4
from bs4 import BeautifulSoup 
!pip install geopy
from geopy.geocoders import Nominatim
!pip install folium
import folium
import io

### Requesting data about neighborhood of Toronto postal code

In [4]:
data_set = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data_set, "html.parser")
soup

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"0d8693ef-728b-48c5-aea3-33763147d7b8","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":1013111980,"wgRevisionId":1013111980,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Wikipedia semi-protected p

### "Cleaning" parsed dataset

In [5]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

print(table_contents)

[{'Postal Code': 'M3A', 'Borough': 'North York', 'Neighborhood': 'Parkwoods'}, {'Postal Code': 'M4A', 'Borough': 'North York', 'Neighborhood': 'Victoria Village'}, {'Postal Code': 'M5A', 'Borough': 'Downtown Toronto', 'Neighborhood': 'Regent Park, Harbourfront'}, {'Postal Code': 'M6A', 'Borough': 'North York', 'Neighborhood': 'Lawrence Manor, Lawrence Heights'}, {'Postal Code': 'M7A', 'Borough': "Queen's Park", 'Neighborhood': 'Ontario Provincial Government'}, {'Postal Code': 'M9A', 'Borough': 'Etobicoke', 'Neighborhood': 'Islington Avenue'}, {'Postal Code': 'M1B', 'Borough': 'Scarborough', 'Neighborhood': 'Malvern, Rouge'}, {'Postal Code': 'M3B', 'Borough': 'North York', 'Neighborhood': 'Don Mills North'}, {'Postal Code': 'M4B', 'Borough': 'East York', 'Neighborhood': 'Parkview Hill, Woodbine Gardens'}, {'Postal Code': 'M5B', 'Borough': 'Downtown Toronto', 'Neighborhood': 'Garden District, Ryerson'}, {'Postal Code': 'M6B', 'Borough': 'North York', 'Neighborhood': 'Glencairn'}, {'Posta

### Transform cleaned and prepared data to frame

In [6]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df = pd.DataFrame(table_contents)
df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [7]:
# lets see size of dataframe
df.shape

(103, 3)

### Add latitude and longitude to frame

In [8]:
url = "http://cocl.us/Geospatial_data"
latlong_data = requests.get(url).content
lat_long = pd.read_csv(io.StringIO(latlong_data.decode("utf-8")))

df = pd.merge(df, lat_long)
df_toronto = df[["Postal Code", "Borough", "Neighborhood", "Latitude", "Longitude"]]
df_toronto

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [9]:
# lets see how much unique borough we have
df_data = df[["Postal Code", "Borough", "Neighborhood", "Latitude", "Longitude"]]
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_toronto['Borough'].unique()),
        df_toronto.shape[0]
    )
)

The dataframe has 15 boroughs and 103 neighborhoods.


In [10]:
# tracking coordinates
address = 'Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Visualize map of Toronto neighborhood

In [11]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, df_toronto in zip(df_toronto['Latitude'],
                                         df_toronto['Longitude'], 
                                         df_toronto['Borough'], 
                                         df_toronto['Neighborhood']):
    label = '{}, {}'.format(df_toronto, borough)  
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Lets take data about one of borough, Downtown Toronto

In [12]:
downtown_data = df_data[df_data['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
5,M6G,Downtown Toronto,Christie,43.669542,-79.422564
6,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
7,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752
8,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576
9,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817


In [13]:
# coordinate of Downtown Toronto
address = 'Downtown Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081164513409.


In [14]:
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(downtown_data['Latitude'], 
                           downtown_data['Longitude'], 
                           downtown_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

### Connect Foursquare for taking data of venues

In [15]:
CLIENT_ID = 'DAZHEWX2X4ACOILUYLMHII0CDVZ5M0CBGLSUDV3PT4IXBYAO' # your Foursquare ID
CLIENT_SECRET = 'PEABAZHM35ODZK1UNGJMCYD02VFBUWFHSKX2CWSHXFC1CA2N' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DAZHEWX2X4ACOILUYLMHII0CDVZ5M0CBGLSUDV3PT4IXBYAO
CLIENT_SECRET:PEABAZHM35ODZK1UNGJMCYD02VFBUWFHSKX2CWSHXFC1CA2N


In [16]:
downtown_data.loc[0, 'Neighborhood']

'Regent Park, Harbourfront'

In [17]:
neighborhood_latitude = downtown_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = downtown_data.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = downtown_data.loc[0, 'Neighborhood'] # neighborhood name
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


In [18]:
# requesting data from foursquare
LIMIT = 50
radius = 300
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    VERSION, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=DAZHEWX2X4ACOILUYLMHII0CDVZ5M0CBGLSUDV3PT4IXBYAO&client_secret=PEABAZHM35ODZK1UNGJMCYD02VFBUWFHSKX2CWSHXFC1CA2N&ll=43.6542599,-79.3606359&v=20180605&radius=300&limit=50'

In [19]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6062d393c8b02727f2ddb072'},
 'response': {'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 13,
  'suggestedBounds': {'ne': {'lat': 43.6569599027, 'lng': -79.35691110008916},
   'sw': {'lat': 43.6515598973, 'lng': -79.36436069991085}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '53b8466a498e83df908c3f21',
       'name': 'Tandem Coffee',
       'location': {'address': '368 King St E',
        'crossStreet': 'at Trinity St',
        'lat': 43.65355870959944,
        'lng': -79.36180945913513,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65355870959944,
          'lng': -79.36180945913513}],
        'distance': 122,
      

In [20]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
# Clean the json and structure it into a dataframe.
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,Tandem Coffee,Coffee Shop,43.653559,-79.361809
1,Roselle Desserts,Bakery,43.653447,-79.362017
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
5,The Extension Room,Gym / Fitness Center,43.653313,-79.359725
6,Buster's Sea Cove Truck,Food Truck,43.653647,-79.357878
7,Enoch Turner Schoolhouse,History Museum,43.652873,-79.361672
8,Sackville Playground,Park,43.654656,-79.359871
9,eco|stems,Flower Shop,43.653536,-79.362016


In [22]:
# returned venues 
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

13 venues were returned by Foursquare.


In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
downtown_venues = getNearbyVenues(names=downtown_data['Neighborhood'],
                                   latitudes=downtown_data["Latitude"],
                                   longitudes=downtown_data['Longitude'])

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [25]:
print(downtown_venues.shape)
downtown_venues

(705, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
...,...,...,...,...,...,...,...
700,Church and Wellesley,43.66586,-79.383160,Rolltation,43.669388,-79.386566,Sushi Restaurant
701,Church and Wellesley,43.66586,-79.383160,The Betty Oliphant Theatre,43.664315,-79.377730,Theater
702,Church and Wellesley,43.66586,-79.383160,Ritz Caribbean Food,43.669545,-79.386506,Caribbean Restaurant
703,Church and Wellesley,43.66586,-79.383160,Tokyo Kitchen,43.668783,-79.385153,Japanese Restaurant


In [26]:
downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,50,50,50,50,50,50
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,50,50,50,50,50,50
Christie,16,16,16,16,16,16
Church and Wellesley,50,50,50,50,50,50
"Commerce Court, Victoria Hotel",50,50,50,50,50,50
"First Canadian Place, Underground city",50,50,50,50,50,50
"Garden District, Ryerson",50,50,50,50,50,50
"Harbourfront East, Union Station, Toronto Islands",50,50,50,50,50,50


In [29]:
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 176 uniques categories.


In [30]:
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
downtown_onehot.shape

(705, 176)

In [32]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,0.000000,...,0.02000,0.000000,0.00,0.00,0.00,0.02,0.000000,0.00,0.00,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.00,0.000000,...,0.00000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000
2,Central Bay Street,0.020000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,0.000000,...,0.02000,0.000000,0.00,0.00,0.00,0.02,0.000000,0.00,0.00,0.000000
3,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,0.000000,...,0.00000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000
4,Church and Wellesley,0.040000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,0.000000,...,0.02000,0.020000,0.02,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000
5,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.04,0.000000,...,0.00000,0.000000,0.00,0.00,0.00,0.02,0.000000,0.00,0.02,0.000000
6,"First Canadian Place, Underground city",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.04,0.000000,...,0.00000,0.000000,0.00,0.00,0.00,0.02,0.000000,0.00,0.02,0.000000
7,"Garden District, Ryerson",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,0.000000,...,0.02000,0.040000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000
8,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,0.000000,...,0.00000,0.020000,0.00,0.00,0.02,0.00,0.000000,0.00,0.00,0.000000
9,"Kensington Market, Chinatown, Grange Park",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,0.000000,...,0.00000,0.000000,0.00,0.00,0.00,0.06,0.000000,0.04,0.02,0.000000


In [33]:
downtown_grouped.shape

(17, 176)

### Data for each neighborhood along with the top 5 most common venues

In [34]:
num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0        Cocktail Bar  0.06
1              Bakery  0.06
2         Coffee Shop  0.06
3  Seafood Restaurant  0.04
4            Beer Bar  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3       Coffee Shop  0.06
4           Airport  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.16
1                Café  0.06
2      Sandwich Place  0.06
3        Burger Joint  0.04
4  Italian Restaurant  0.04


----Christie----
                venue  freq
0       Grocery Store  0.25
1                Café  0.19
2                Park  0.12
3  Italian Restaurant  0.06
4         Candy Store  0.06


----Church and Wellesley----
                 venue  freq
0     Sushi Restaurant  0.08
1          Coffee Shop  0.06
2  Japanese Restaurant  0.06
3       

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Cocktail Bar,Bakery,Coffee Shop,Seafood Restaurant,Beer Bar,Farmers Market,Pharmacy,Restaurant,Cheese Shop,Fountain
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Airport,Sculpture Garden,Bar,Boutique,Harbor / Marina,Rental Car Location
2,Central Bay Street,Coffee Shop,Café,Sandwich Place,Burger Joint,Italian Restaurant,Yoga Studio,Discount Store,Bubble Tea Shop,Chinese Restaurant,Ramen Restaurant
3,Christie,Grocery Store,Café,Park,Italian Restaurant,Candy Store,Nightclub,Athletics & Sports,Restaurant,Baby Store,Coffee Shop
4,Church and Wellesley,Sushi Restaurant,Coffee Shop,Japanese Restaurant,Yoga Studio,Gay Bar,Restaurant,Men's Store,Bookstore,Diner,Breakfast Spot
5,"Commerce Court, Victoria Hotel",Hotel,Café,Coffee Shop,Restaurant,Gym,Seafood Restaurant,Deli / Bodega,Beer Bar,Gastropub,American Restaurant
6,"First Canadian Place, Underground city",Café,Coffee Shop,Restaurant,Hotel,Seafood Restaurant,Deli / Bodega,Concert Hall,Gastropub,American Restaurant,Gym
7,"Garden District, Ryerson",Coffee Shop,Café,Clothing Store,Ramen Restaurant,Theater,Bookstore,Cosmetics Shop,Middle Eastern Restaurant,Mexican Restaurant,Beer Bar
8,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Aquarium,Hotel,Brewery,Park,Plaza,Café,Chinese Restaurant,Bistro,Bubble Tea Shop
9,"Kensington Market, Chinatown, Grange Park",Café,Mexican Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Gaming Cafe,Vietnamese Restaurant,Farmers Market,Cheese Shop,Brewery,Food Court


### Cluster Neighborhoods

In [37]:
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/srv/conda/envs/notebook/lib/python3.7/site-

array([4, 2, 4, 3, 4, 0, 0, 4, 4, 4], dtype=int32)

In [38]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_data

# merge downtown_grouped with manhattan_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head() 

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Park,Bakery,Café,Theater,Breakfast Spot,Farmers Market,Chocolate Shop,Cosmetics Shop
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,4,Coffee Shop,Café,Clothing Store,Ramen Restaurant,Theater,Bookstore,Cosmetics Shop,Middle Eastern Restaurant,Mexican Restaurant,Beer Bar
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,4,Café,Cosmetics Shop,Coffee Shop,Gastropub,Park,Creperie,Seafood Restaurant,Farmers Market,Hotel,Middle Eastern Restaurant
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,4,Cocktail Bar,Bakery,Coffee Shop,Seafood Restaurant,Beer Bar,Farmers Market,Pharmacy,Restaurant,Cheese Shop,Fountain
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,4,Coffee Shop,Café,Sandwich Place,Burger Joint,Italian Restaurant,Yoga Studio,Discount Store,Bubble Tea Shop,Chinese Restaurant,Ramen Restaurant


### Create map for visualize clustering

In [41]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], 
                                  downtown_merged['Longitude'], 
                                  downtown_merged['Neighborhood'], 
                                  downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Finally, examine Clusters

In [42]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Pub,Park,Bakery,Café,Theater,Breakfast Spot,Farmers Market,Chocolate Shop,Cosmetics Shop
6,Downtown Toronto,0,Café,Coffee Shop,Restaurant,Steakhouse,Concert Hall,Pizza Place,Gluten-free Restaurant,Bookstore,Seafood Restaurant,Burrito Place
8,Downtown Toronto,0,Coffee Shop,Café,Seafood Restaurant,Restaurant,Hotel,Beer Bar,Japanese Restaurant,French Restaurant,Plaza,Speakeasy
9,Downtown Toronto,0,Hotel,Café,Coffee Shop,Restaurant,Gym,Seafood Restaurant,Deli / Bodega,Beer Bar,Gastropub,American Restaurant
15,Downtown Toronto,0,Café,Coffee Shop,Restaurant,Hotel,Seafood Restaurant,Deli / Bodega,Concert Hall,Gastropub,American Restaurant,Gym


In [52]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Rosedale,1,Park,Playground,Trail,Museum,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant


In [49]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,"CN Tower, King and Spadina, Railway Lands, Har...",2,Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Airport,Sculpture Garden,Bar,Boutique,Harbor / Marina,Rental Car Location


In [48]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Christie,3,Grocery Store,Café,Park,Italian Restaurant,Candy Store,Nightclub,Athletics & Sports,Restaurant,Baby Store,Coffee Shop


In [45]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Garden District, Ryerson",4,Coffee Shop,Café,Clothing Store,Ramen Restaurant,Theater,Bookstore,Cosmetics Shop,Middle Eastern Restaurant,Mexican Restaurant,Beer Bar
2,St. James Town,4,Café,Cosmetics Shop,Coffee Shop,Gastropub,Park,Creperie,Seafood Restaurant,Farmers Market,Hotel,Middle Eastern Restaurant
3,Berczy Park,4,Cocktail Bar,Bakery,Coffee Shop,Seafood Restaurant,Beer Bar,Farmers Market,Pharmacy,Restaurant,Cheese Shop,Fountain
4,Central Bay Street,4,Coffee Shop,Café,Sandwich Place,Burger Joint,Italian Restaurant,Yoga Studio,Discount Store,Bubble Tea Shop,Chinese Restaurant,Ramen Restaurant
7,"Harbourfront East, Union Station, Toronto Islands",4,Coffee Shop,Aquarium,Hotel,Brewery,Park,Plaza,Café,Chinese Restaurant,Bistro,Bubble Tea Shop
10,"University of Toronto, Harbord",4,Café,Bakery,Italian Restaurant,Japanese Restaurant,Bookstore,Bar,Pub,Beer Bar,Beer Store,Sandwich Place
11,"Kensington Market, Chinatown, Grange Park",4,Café,Mexican Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Gaming Cafe,Vietnamese Restaurant,Farmers Market,Cheese Shop,Brewery,Food Court
14,"St. James Town, Cabbagetown",4,Bakery,Coffee Shop,Pizza Place,Italian Restaurant,Restaurant,Pub,Café,Indian Restaurant,Liquor Store,Pet Store
16,Church and Wellesley,4,Sushi Restaurant,Coffee Shop,Japanese Restaurant,Yoga Studio,Gay Bar,Restaurant,Men's Store,Bookstore,Diner,Breakfast Spot
